In [172]:
from sklearn.feature_extraction.text import *
##CountVectorizer,TfidfVectorizer 사용, 특성추출
#CountVectorizer : 각 텍스트에서 단어 출현 횟수를 카운팅한 벡터
#TfidfVectorizer : TF-IDF라는 값을 사용하여 CountVectorizer의 단점을 보완함
#                   빈도수와 역빈도수를 함께 고려
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [173]:
spam_df = pd.read_csv('./spamData/spam.csv', encoding='latin1')

In [174]:
spam_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [175]:
spam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.8+ KB


### extraction!(reg) => not null, word

In [176]:
#결측치를 빼보자.
spam_df = spam_df[spam_df['v2'].notnull()]
spam_df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [177]:
spam_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 261.2+ KB


In [178]:
#필요없는 컬럼을 지워보자.
spam_df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],
             inplace=True)

In [179]:
#중복값 제거
spam_df.drop_duplicates(subset=['v2'], inplace=True)
spam_df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [180]:
spam_df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [181]:
#정규식을 사용해서 영어제외 전부 없애보자.(특수문자 등)


spam_df['v2'] = spam_df['v2'].apply(lambda x: re.sub(r'[^a-zA-Z]+'," ",x))
spam_df.head()

,v1,v2
0,ham,Go until jurong point crazy Available only in ...
1,ham,Ok lar Joking wif u oni
2,spam,Free entry in a wkly comp to win FA Cup final ...
3,ham,U dun say so early hor U c already then say
4,ham,Nah I don t think he goes to usf he lives arou...


### classification => train/test data

In [182]:
#test,train데이터를 나누어보자.

In [183]:
spam_df.iloc[:,1]

0       Go until jurong point crazy Available only in ...
1                                Ok lar Joking wif u oni 
2       Free entry in a wkly comp to win FA Cup final ...
3            U dun say so early hor U c already then say 
4       Nah I don t think he goes to usf he lives arou...
                              ...                        
5567    This is the nd time we have tried contact u U ...
5568                   Will b going to esplanade fr home 
5569    Pity was in mood for that So any other suggest...
5570    The guy did some bitching but I acted like i d...
5571                            Rofl Its true to its name
Name: v2, Length: 5169, dtype: object

In [184]:
X_train, X_test, y_train, y_test = train_test_split(spam_df.iloc[:,1],#문제 
                                                    spam_df.iloc[:,0],#정답 
                                                    test_size=0.4, 
                                                    random_state=42)

In [185]:
print('X_train: ',X_train.shape, 'y_train: ',y_train.shape, 'X_test: ',X_test.shape,'y_test: ',y_test.shape)

X_train:  (3101,) y_train:  (3101,) X_test:  (2068,) y_test:  (2068,)


### stopwords & vectorize => choice(Counter, Tf-idf) : fit(train) —> transform(train,test)
### 1. 사전을 만들어라 => fit()
### 2. 사전에 들어있는 id로 인코딩을 해라! => transform()
### 3. 1번2번을 함께 처리하고 싶은 경우 => fit_transform()

In [186]:
tfidf = TfidfVectorizer(min_df=5,
                        lowercase = True, 
                        stop_words='english')

In [187]:
tfidf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [188]:
#fit한 것을 transform 하자.
X_train =  tfidf.transform(X_train)
X_test= tfidf.transform(X_test)

In [189]:
print(X_train.shape, X_test.shape)

(3101, 928) (2068, 928)


In [190]:
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [191]:
cv.vocabulary_

{'need': 3369,
 'you': 5896,
 'to': 5230,
 'be': 452,
 'in': 2460,
 'my': 3320,
 'strong': 4880,
 'arms': 270,
 'haha': 2166,
 'heard': 2237,
 'that': 5122,
 'text': 5102,
 'me': 3083,
 'when': 5690,
 're': 4089,
 'around': 273,
 'what': 5686,
 'love': 2933,
 'hear': 2236,
 'see': 4421,
 'sundayish': 4947,
 'then': 5134,
 'pete': 3743,
 'is': 2550,
 'this': 5155,
 'your': 5902,
 'phone': 3756,
 'still': 4843,
 'its': 2567,
 'jenny': 2605,
 'from': 1945,
 'college': 977,
 'and': 181,
 'leanne': 2788,
 'are': 255,
 'up': 5436,
 'now': 3467,
 'doing': 1390,
 'nothing': 3457,
 'not': 3454,
 'having': 2223,
 'dinner': 1331,
 'us': 5463,
 'jamster': 2586,
 'get': 2020,
 'free': 1915,
 'wallpaper': 5591,
 'heart': 2238,
 'apply': 232,
 'only': 3553,
 'help': 2256,
 'call': 722,
 'sure': 4964,
 'yet': 5884,
 'trying': 5334,
 'hold': 2307,
 'of': 3507,
 'him': 2282,
 'cheers': 859,
 'for': 1880,
 'the': 5124,
 'message': 3125,
 'zogtorius': 5927,
 've': 5506,
 'been': 472,
 'staring': 4810,
 'a

In [192]:
tfidf.inverse_transform(X_train)

[array(['road', 'pple', 'oredi', 'old', 'lot', 'got', 'eat'], dtype='<U15'),
 array(['right'], dtype='<U15'),
 array(['thanks', 'stuff', 'service', 'rakhesh', 'leaving', 'hi'],
       dtype='<U15'),
 array(['week', 'weed', 'news', 'money'], dtype='<U15'),
 array(['snow', 'home', 'enjoy'], dtype='<U15'),
 array(['want', 'today', 'text', 'right', 'gonna', 'come', 'check'],
       dtype='<U15'),
 array(['unlimited', 'txt', 'stop', 'msg', 'mobile', 'hrs', 'free',
        'direct', 'day', 'access'], dtype='<U15'),
 array(['wit', 'msgs', 'just', 'getting', 'dis', 'different'], dtype='<U15'),
 array(['yeah', 'thought', 'lemme', 'later', 'know', 'goin'], dtype='<U15'),
 array(['gone'], dtype='<U15'),
 array(['ve', 'jay', 'got'], dtype='<U15'),
 array(['ve', 'money', 'make', 'let', 'know', 'got', 'carlos'],
       dtype='<U15'),
 array(['way', 'da', 'come'], dtype='<U15'),
 array(['sit', 'na', 'happy', 'da'], dtype='<U15'),
 array(['bit', 'alright'], dtype='<U15'),
 array(['touch', 'new', 'mean

In [193]:
tfidf.inverse_transform(X_test)

[array(['download', 'did'], dtype='<U15'),
 array(['wid', 'wat', 'wana', 'ur', 'swt', 'smile', 'red', 'plz', 'pass',
        'orange', 'miss', 'luv', 'hot', 'face', 'dis', 'blue'],
       dtype='<U15'),
 array(['ok'], dtype='<U15'),
 array(['later', 'ill', 'film'], dtype='<U15'),
 array(['sorry', 'phone', 'ok', 'left', 'date'], dtype='<U15'),
 array(['urgent', 'rate', 'prize', 'nd', 'national', 'mobile', 'contact',
        'caller', 'bt', 'box', 'bonus', 'awarded', 'attempt'], dtype='<U15'),
 array(['school', 'plus', 'offer', 'got'], dtype='<U15'),
 array(['week', 'ringtone', 'new', 'mobile', 'gr', 'direct', 'club'],
       dtype='<U15'),
 array(['week', 'know', 'going', 'don', 'da'], dtype='<U15'),
 array(['started', 'office', 'da'], dtype='<U15'),
 array(['waiting', 'ur', 'test', 'tell', 'reply', 'number', 'lt', 'gt',
        'face'], dtype='<U15'),
 array(['set', 'big'], dtype='<U15'),
 array(['wow', 'win', 'uk', 'txt', 'tickets', 'info', 'club', 'book'],
       dtype='<U15'),
 arra

### ml model fit(train) => predict(test) => accuracy check

In [194]:
SA_lr = LogisticRegression(random_state = 0)

In [195]:
SA_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [196]:
test_predict = SA_lr.predict(X_test)

In [197]:
print('스팸 메일 분석 정확도 : ', round(accuracy_score(y_test, test_predict), 5))

스팸 메일 분석 정확도 :  0.96277


### one mail => predict!!

In [198]:
#v2컬럼의 20번째 행이 spam인지 ham 인지 판별해보자.!
sp = spam_df['v2'][20]

In [199]:
sp2 = re.sub(r'[^[a-zA-Z]*$', "", sp)
sp2

'Is that seriously how you spell his name'

In [200]:
sp_tfidf = tfidf.transform([sp2])
sp_tfidf

<1x928 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [201]:
sp2_predict = SA_lr.predict(sp_tfidf)
sp2_predict, sp2_predict[0]

(array(['ham'], dtype=object), 'ham')

In [202]:
if(sp2_predict[0] == 'spam'):
    print(sp2, '==> spam mail')
else:
    print(sp2, '==> ham mail')

Is that seriously how you spell his name ==> ham mail


### 모델 저장

In [203]:
import pickle
import joblib

In [204]:
tfidf_fit = tfidf.fit(spam_df['v2'])
tfidf_fit

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [205]:
saved_model = pickle.dumps(SA_lr)

In [206]:
lr_from_pickle = pickle.loads(saved_model)
lr_from_pickle.predict(sp_tfidf[0])

array(['ham'], dtype=object)

In [207]:
joblib.dump(SA_lr, 'spam_SA_lr.pkl')

['spam_SA_lr.pkl']

In [208]:
lr_from_joblib = joblib.load('spam_SA_lr.pkl')
lr_from_joblib

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [209]:
lr_from_joblib.predict(sp_tfidf[0])

array(['ham'], dtype=object)